In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict
import uuid
import json

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")



pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210601-1303-0.2.62-84fa81b9ea3d.log


In [2]:
run_hash="94f5cc00"

ht=hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_sanger_cohorts_new_SYNONYMOUS.ht')
de_novo=hl.read_table(f'{lustre_dir}/Sanger_cohort_denovo_table.ht')
family_stats=hl.read_table(f'{lustre_dir}/variant_qc/Sanger_cohorts_family_stats.ht')

In [3]:
family_stats.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'fam_id': str 
    'errors': int64 
    'snp_errors': int64 
----------------------------------------
Key: ['s']
----------------------------------------


In [4]:
ht=ht.annotate(de_novo_data=de_novo[ht.key].de_novo_data)



In [5]:
ht.write(f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_sanger_cohorts_new_SYNONYMOUS_denovo.ht')

2021-06-01 11:04:28 Hail: INFO: wrote table with 28008043 rows in 487 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/94f5cc00/rf_result_sanger_cohorts_new_SYNONYMOUS_denovo.ht
    Total size: 3.11 GiB
    * Rows: 3.11 GiB
    * Globals: 292.00 B
    * Smallest partition: 10484 rows (1.25 MiB)
    * Largest partition:  58824 rows (7.68 MiB)


In [7]:
de_novo_table = de_novo_table.key_by(
        'locus', 'alleles').collect_by_key('de_novo_data')

In [21]:
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}/rf_result_sanger_cohorts_new_SYNONYMOUS_denovo.ht')

In [23]:
ht_fs=hl.read_table(f'{lustre_dir}/variant_qc/Sanger_cohorts_family_stats.ht')

In [24]:
ht_fs.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'family_stats': array<struct {
        mendel: struct {
            errors: int64
        }, 
        tdt: struct {
            t: int64, 
            u: int64, 
            chi_sq: float64, 
            p_value: float64
        }, 
        unrelated_qc_callstats: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            homozygote_count: array<int32>
        }, 
        meta: dict<str, str>
    }> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [25]:
ht=ht.annotate(family_stats=ht_fs[ht.key].family_stats)

In [27]:
ht.write(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_sanger_cohorts_new_SYNONYMOUS_denovo_family_stats.ht')

2021-06-01 13:28:35 Hail: INFO: wrote table with 28008043 rows in 487 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/94f5cc00/rf_result_sanger_cohorts_new_SYNONYMOUS_denovo_family_stats.ht
    Total size: 3.45 GiB
    * Rows: 3.45 GiB
    * Globals: 292.00 B
    * Smallest partition: 10484 rows (1.25 MiB)
    * Largest partition:  58824 rows (9.52 MiB)
2021-06-01 13:28:35 Hail: INFO: wrote table with 28008043 rows in 487 partitions to file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/models/94f5cc00_rf_result_sanger_cohorts_new_SYNONYMOUS_denovo_family_stats.ht
    Total size: 3.45 GiB
    * Rows: 3.45 GiB
    * Globals: 292.00 B
    * Smallest partition: 10484 rows (1.25 MiB)
    * Largest partition:  58824 rows (9.52 MiB)


In [28]:
ht=hl.read_table(f'{lustre_dir}/variant_qc/models/{run_hash}_rf_result_sanger_cohorts_new_SYNONYMOUS_denovo_family_stats.ht')

In [29]:
ht.describe()

----------------------------------------
Global fields:
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'a_index': int32 
    'was_split': bool 
    'AC': array<int32> 
    'AF': array<float64> 
    'AN': int32 
    'AS_BaseQRankSum': array<float64> 
    'AS_FS': array<float64> 
    'AS_InbreedingCoeff': array<float64> 
    'AS_MQ': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'BaseQRankSum': float64 
    'DB': bool 
    'DP': int32 
    'DS': bool 
    'END': int32 
    'ExcessHet': float64 
    'FS': float64 
    'InbreedingCoeff': float32 
    '